# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [3]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [4]:
cur.execute('''SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [3]:
# Your code here; use a subquery. No join is necessary 
cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
                FROM customers
                WHERE customerNumber IN
                    (SELECT customerNumber FROM orders WHERE orderDate = "2003-01-31")''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
cur.execute('''SELECT productName, COUNT(orderNumber) as numOrders
                FROM products
                JOIN orderdetails USING(productCode)
                GROUP BY productName
                ORDER BY numOrders DESC''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,productName,numOrders
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


In [4]:
# I'm not sure if this one is right.


# Your code here

# Hard to think in contrived subqueries where they aren't necessary

cur.execute('''SELECT productName, COUNT(orderNumber) as numOrders
                FROM (SELECT productName, orderNumber
                        from products
                        JOIN orderdetails USING(productCode))
                        
                GROUP BY productName
                ORDER BY numOrders DESC''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


,productName,numOrders
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


In [24]:
cur.execute('''SELECT productCode, productName FROM products ORDER BY productName''')
cur.fetchall()

[('S18_3136', '18th Century Vintage Horse Carriage'),
 ('S24_2011', '18th century schooner'),
 ('S24_2841', '1900s Vintage Bi-Plane'),
 ('S24_4278', '1900s Vintage Tri-Plane'),
 ('S18_3140', '1903 Ford Model A'),
 ('S18_4522', '1904 Buick Runabout'),
 ('S18_2248', '1911 Ford Town Car'),
 ('S24_3151', '1912 Ford Model T Delivery Wagon'),
 ('S18_2949', '1913 Ford Model T Speedster'),
 ('S18_1749', '1917 Grand Touring Sedan'),
 ('S18_3320', '1917 Maxwell Touring Car'),
 ('S18_2432', '1926 Ford Fire Engine'),
 ('S24_1785', '1928 British Royal Navy Airplane'),
 ('S32_4289', '1928 Ford Phaeton Deluxe'),
 ('S18_2795', '1928 Mercedes-Benz SSK'),
 ('S50_1341', '1930 Buick Marquette Phaeton'),
 ('S18_4409', '1932 Alfa Romeo 8C2300 Spider Sport'),
 ('S18_2325', '1932 Model A Ford J-Coupe'),
 ('S18_2957', '1934 Ford V8 Coupe'),
 ('S24_4258', '1936 Chrysler Airflow'),
 ('S18_2625', '1936 Harley Davidson El Knucklehead'),
 ('S24_3969', '1936 Mercedes Benz 500k Roadster'),
 ('S18_1367', '1936 Mercede

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [40]:
cur.execute('''SELECT productName, num_customers
                FROM products
                JOIN
                    (SELECT productCode, COUNT(customerNumber) as num_customers
                    FROM orderdetails
                    JOIN orders USING(orderNumber)
                    GROUP BY productCode)
                    USING(productCode)
                GROUP BY productName
                ORDER BY num_customers DESC''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description]).head()


,productName,num_customers
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28


### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [46]:
# Select the Product Name and the Total Number of People Who Have Ordered Each Product¶
# Sort the results in descending order.

# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

# I don't seen the need for DISTINCT here.  We are counting the customerNumbers when grouped by customerNumber to
# tell us the number of customers for each product

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


110

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [71]:
# Your code here
# cur.execute('''SELECT employeeNumber, contactFirstName, contactLastName, offices.city, oficeCode''')

# # products and customers that ordered them
# cur.execute('''SELECT DISTINCT customerNumber, productCode
#                 FROM orders
#                 JOIN orderdetails USING(orderNumber)''')

# # count the distinct customers for each product and filter by num_customers < 20
# cur.execute('''SELECT productCode, COUNT(customerNumber) as num_customers
#                 FROM
#                     -- Unique customer product code relationships to avoid overcouting
#                     (SELECT DISTINCT customerNumber, productCode
#                     FROM orders
#                     JOIN orderdetails USING(orderNumber))
#                 GROUP BY productCode HAVING num_customers < 20''')

# # select unfilterd reslts with product ALL product Codes
# cur.execute('''SELECT employeeNumber, firstName, lastName, offices.city as officeCity, officeCode, productCode
#                 FROM employees
#                 JOIN offices USING(officeCode)
#                 JOIN customers ON employeeNumber == salesRepEmployeeNumber
#                 JOIN orders USING(customerNumber)
#                 JOIN orderdetails USING(orderNumber)''')

# filter employees by only the products in the first list (count < 20)
cur.execute('''SELECT employeeNumber, firstName, lastName, offices.city as officeCity, officeCode /*, productCode */
                FROM employees
                JOIN offices USING(officeCode)
                JOIN customers ON employeeNumber == salesRepEmployeeNumber
                JOIN orders USING(customerNumber)
                JOIN orderdetails USING(orderNumber)
                WHERE productCode IN
                    (SELECT productCode
                    FROM
                        -- Unique customer product code relationships to avoid overcouting
                        (SELECT DISTINCT customerNumber, productCode
                        FROM orders
                        JOIN orderdetails USING(orderNumber))
                    GROUP BY productCode HAVING COUNT(customerNumber) < 20)''')
pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


,employeeNumber,firstName,lastName,officeCity,officeCode
0,1370,Gerard,Hernandez,Paris,4
1,1501,Larry,Bott,London,7
2,1501,Larry,Bott,London,7
3,1501,Larry,Bott,London,7
4,1501,Larry,Bott,London,7
...,...,...,...,...,...
122,1611,Andy,Fixter,Sydney,6
123,1611,Andy,Fixter,Sydney,6
124,1611,Andy,Fixter,Sydney,6
125,1611,Andy,Fixter,Sydney,6



## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [80]:
# Your code here
# Customers with Average Credit Limit above 15K

# No subqueries are necessary.

cur.execute('''SELECT employeeNumber, firstName, lastName, COUNT(customerNumber), AVG(creditLimit)
                FROM employees
                JOIN customers ON employeeNumber = salesRepEmployeeNumber
                GROUP BY employeeNumber
                HAVING AVG(creditLimit) > 15000''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


,employeeNumber,firstName,lastName,COUNT(customerNumber),AVG(creditLimit)
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!